# Principais métodos de chains com LCEL

A interface padrão do LCEL inclui os seguintes métodos:

• stream : transmitir de volta fragmentos da resposta

• invoke : chamar a cadeia com um input

• batch : chamar a cadeia com uma lista de inputs


Esses também possuem métodos assíncronos correspondentes que devem ser usados com a sintaxe asyncio await para concorrência:

• astream : transmitir de volta fragmentos de resposta de forma assíncrona.

• ainvoke : chamar a cadeia com um input de forma assíncrona.

• abatch : chamar a cadeia com uma lista de input de forma assíncrona.

• astream_log : transmitir de volta etapas intermediárias à medida que acontecem, além da resposta final.

• astream_events : transmitir eventos beta à medida que acontecem na cadeia.

In [14]:
from pyexpat import model
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(model='gpt-3.5-turbo')
prompt = ChatPromptTemplate.from_template("Crie um parágrafo filosofando sobre o seguinte assunto: {subject}")

chain = prompt | model 

## INVOKE

O invoke é o método básico para uma input da caideia receber uma resposta

In [15]:
chain.invoke({"subject": "O que é a vida?"})

AIMessage(content='A vida é um mistério envolto em fragilidade e beleza. É um ciclo contínuo de nascimentos, mortes e renascimentos, onde cada momento é único e precioso. É uma jornada repleta de desafios, aprendizados e oportunidades de crescimento. A vida é um presente que nos foi dado, cheio de possibilidades e potenciais. É a oportunidade de experimentar, evoluir e buscar a felicidade em meio às adversidades e incertezas. Cada um de nós tem o poder de dar significado e propósito à nossa existência, transformando a vida em uma obra de arte em constante construção.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 152, 'prompt_tokens': 31, 'total_tokens': 183, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason

## STREAM

Para recebermos uma saída conforme ela é gerada pelo modelo utilizando o stream

In [19]:
for chunk in chain.stream({'subject':'cachorrinho'}):
    print(chunk.content, end = '', flush=True)

O cachorrinho, pequeno em tamanho mas gigante em amor, nos ensina lições valiosas sobre lealdade, companheirismo e alegria. Sua presença em nossas vidas traz uma sensação de paz e conforto que nos faz sentir amados e protegidos. Eles nos mostram que, mesmo nos momentos mais difíceis, sempre haverá alguém ao nosso lado para nos apoiar e nos fazer sorrir. O cachorrinho é um exemplo de pureza e simplicidade, nos lembrando que a felicidade está nas pequenas coisas e que o amor verdadeiro não precisa de palavras, apenas de um olhar ou um carinho. Em sua inocência, nos mostram que a vida pode ser mais simples e mais bonita, se aprendermos a valorizar as pequenas coisas e a compartilhar o amor incondicional que eles nos oferecem.

## BATCH

Para fazermos múltiplas requisições em paralelo utilizamos o batch.

In [24]:
SUBJECTS = [
    {"subject": "O que é a vida?"},
    {"subject": "O que é a morte?"},
    {'subject': 'O que é a felicidade?'},
    {'subject': 'O que é a tristeza?'},
    {'subject': 'O que é a arte?'},
    {'subject': 'O que é a ciência?'},
    {'subject': 'O que é a religião?'},
    {'subject': 'O que é a política?'},

]

chain.batch(SUBJECTS, config={'max_concurrency': 2})

[AIMessage(content='A vida é um enigma que nos intriga desde o princípio dos tempos. É um ciclo constante de nascimentos, mortes, amores, dores e aprendizados. É a capacidade de sentir, de amar, de evoluir, de se adaptar às adversidades e de encontrar sentido em meio ao caos. A vida é uma jornada de descobertas e experiências, onde cada indivíduo é único em sua trajetória e em sua busca por significado. É a oportunidade de viver intensamente, de se conectar com o universo e com os outros seres que compartilham esse mesmo caminho. É um presente precioso e fugaz, que devemos valorizar e aproveitar ao máximo, pois nunca sabemos quando ela pode chegar ao fim.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 174, 'prompt_tokens': 31, 'total_tokens': 205, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_to

## AInvoke

Todos os métodos possuem assíncronos correspondentes que devem ser usados com a sintaxe asyncio await para concorrência.

In [31]:
import asyncio

async def processa_chain(input):
    resposta = await chain.ainvoke(input)
    return resposta

task1 = asyncio.create_task(processa_chain({'subject': 'O que é a vida?'}))
task2 = asyncio.create_task(processa_chain({'subject': 'O que é a amar?'}))
task3 = asyncio.create_task(processa_chain({'subject': 'O que é a pensar?'}))
task4 = asyncio.create_task(processa_chain({'subject': 'O que é a arte?'}))
task5 = asyncio.create_task(processa_chain({'subject': 'O que é a ciência?'}))
task6 = asyncio.create_task(processa_chain({'subject': 'O que é a religião?'}))

await task1
await task2
await task3
await task4
await task5


AIMessage(content='A ciência é um conjunto de conhecimentos sistematizados e fundamentados em evidências empíricas, que busca compreender a natureza e o funcionamento do mundo ao nosso redor. É por meio da ciência que buscamos desvendar os mistérios da vida, do universo e de tudo o que nos rodeia. Ela nos permite questionar, experimentar, observar, analisar e compreender de forma racional e metódica os fenômenos que nos cercam. A ciência é um caminho de descoberta, um instrumento de transformação e um meio de desenvolvimento intelectual e tecnológico. É um esforço contínuo em busca da verdade, da objetividade e do conhecimento, que nos leva a repensar constantemente nossas ideias e conceitos, ampliando nossa compreensão do mundo e de nós mesmos. Assim, a ciência se revela como uma das maiores e mais poderosas ferramentas do ser humano para desvendar os mistérios do universo e avançar no progresso da humanidade.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'

In [26]:
task1.result()

AIMessage(content='A vida é um enigma que permeia a existência de todos os seres vivos neste vasto universo. Ela é um ciclo contínuo de nascimento, crescimento, aprendizado, desafios e transformações. É um dom precioso que nos foi concedido, repleto de momentos de alegria, tristeza, amor, perdas e conquistas. A vida é uma jornada cheia de incertezas, onde cada um de nós busca encontrar o seu propósito e significado. Ela é efêmera, fugaz e frágil, por isso devemos valorizá-la e vivê-la da melhor forma possível. A vida é uma dádiva que nos permite experienciar as mais diversas emoções e sensações, nos ensinando lições importantes e nos fazendo evoluir como seres humanos. É nosso dever apreciar cada instante, cada sorriso, cada lágrima, cada abraço, pois a vida é única e precisa ser vivida intensamente.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 227, 'prompt_tokens': 31, 'total_tokens': 258, 'completion_tokens_details': {'accepted_predic

In [29]:
task2.result()

AIMessage(content='Amar é mais do que um simples sentimento, é uma entrega total de si mesmo ao outro. É se doar sem esperar nada em troca, é aceitar as imperfeições do outro e ainda assim enxergar sua beleza interior. É querer o bem do outro acima de tudo, é estar presente nos momentos felizes e também nos momentos difíceis. Amar é compreender, é respeitar, é cuidar, é construir uma parceria baseada na confiança e no respeito mútuo. É encontrar a felicidade na felicidade do outro e ser capaz de perdoar e recomeçar quantas vezes forem necessárias. Amar é um ato de coragem e de generosidade, é a essência da vida e a fonte de toda a felicidade verdadeira.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 188, 'prompt_tokens': 32, 'total_tokens': 220, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_toke

In [30]:
task3.result()

AIMessage(content='Pensar é uma capacidade exclusivamente humana que nos permite refletir, analisar, questionar e criar. É através do pensamento que formamos nossas ideias, opiniões e conceitos sobre o mundo que nos cerca. Pensar vai além do simples ato de processar informações, envolve uma interpretação única e pessoal. É o que nos diferencia de outras espécies e nos permite evoluir, inovar e buscar respostas para as grandes questões da vida. Pensar é um exercício constante que nos desafia a expandir nossos horizontes e a enxergar além do óbvio. É através do pensamento crítico e reflexivo que somos capazes de transformar o mundo e a nós mesmos.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 170, 'prompt_tokens': 31, 'total_tokens': 201, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}

In [ ]:
from time import time

a = time()

task1 = asyncio.create_task(processa_chain({'subject': 'O que é a vida?'}))
task2 = asyncio.create_task(processa_chain({'subject': 'O que é a amar?'}))
task3 = asyncio.create_task(processa_chain({'subject': 'O que é a pensar?'}))
task4 = asyncio.create_task(processa_chain({'subject': 'O que é a arte?'}))
task5 = asyncio.create_task(processa_chain({'subject': 'O que é a ciência?'}))
task6 = asyncio.create_task(processa_chain({'subject': 'O que é a religião?'}))

await task1
await task2
await task3
await task4
await task5

b = time()
print(b-a)

3.464428663253784


In [ ]:
from time import time

a = time()

task1 = chain.invoke(processa_chain({'subject': 'O que é a vida?'}))
task2 = chain.invoke(processa_chain({'subject': 'O que é a amar?'}))
task3 = chain.invoke(processa_chain({'subject': 'O que é a pensar?'}))
task4 = chain.invoke(processa_chain({'subject': 'O que é a arte?'}))
task5 = chain.invoke(processa_chain({'subject': 'O que é a ciência?'}))
task6 = chain.invoke(processa_chain({'subject': 'O que é a religião?'}))

b = time()
print(b-a)


C:\Users\edinocencio\AppData\Local\Temp\ipykernel_53612\1808015849.py:5: RuntimeWarning: coroutine 'processa_chain' was never awaited
  task1 = chain.invoke(processa_chain({'subject': 'O que é a vida?'}))
C:\Users\edinocencio\AppData\Local\Temp\ipykernel_53612\1808015849.py:6: RuntimeWarning: coroutine 'processa_chain' was never awaited
  task2 = chain.invoke(processa_chain({'subject': 'O que é a amar?'}))
C:\Users\edinocencio\AppData\Local\Temp\ipykernel_53612\1808015849.py:7: RuntimeWarning: coroutine 'processa_chain' was never awaited
  task3 = chain.invoke(processa_chain({'subject': 'O que é a pensar?'}))
C:\Users\edinocencio\AppData\Local\Temp\ipykernel_53612\1808015849.py:8: RuntimeWarning: coroutine 'processa_chain' was never awaited
  task4 = chain.invoke(processa_chain({'subject': 'O que é a arte?'}))
C:\Users\edinocencio\AppData\Local\Temp\ipykernel_53612\1808015849.py:9: RuntimeWarning: coroutine 'processa_chain' was never awaited
  task5 = chain.invoke(processa_chain({'subj

17.48121929168701


C:\Users\edinocencio\AppData\Local\Temp\ipykernel_53612\1808015849.py:10: RuntimeWarning: coroutine 'processa_chain' was never awaited
  task6 = chain.invoke(processa_chain({'subject': 'O que é a religião?'}))
